In [ ]:
!pip install -q keybert sentence-transformers nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.3 MB/s eta 0:00:00


In [ ]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from IPython.display import display
from google.colab import files

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

MODEL INITIALIZATION


In [ ]:
# Load embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
kw_model = KeyBERT(model=embedding_model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Upload transcript
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving Lecture 10 - Neural Networks.txt to Lecture 10 - Neural Networks (1).txt


UNIQUE KEYWORD CHECKER

In [ ]:
# Utility function: check if keyword is semantically similar to existing ones
def is_unique(keyword, existing_embeddings, threshold=0.8):
    keyword_emb = embedding_model.encode([keyword])
    if not existing_embeddings:
        return True
    sims = cosine_similarity(keyword_emb, existing_embeddings)
    max_sim = sims.max()
    return max_sim < threshold

SENTENCE WISE CHUNKS

In [ ]:
# Read transcript
with open(filename, 'r') as f:
    transcript = f.read()

# Split into sentences
sentences = sent_tokenize(transcript)

# Initialize tracking
unique_keywords = []
unique_keyword_embeddings = []
steps = []
unique_counts = []

# Process each sentence
for i, sent in enumerate(tqdm(sentences, desc="Processing Sentences")):
    keywords = kw_model.extract_keywords(sent, top_n=5, stop_words='english')
    for kw, _ in keywords:
        if is_unique(kw, unique_keyword_embeddings):
            unique_keywords.append(kw)
            unique_keyword_embeddings.append(embedding_model.encode(kw))
    steps.append(i + 1)
    unique_counts.append(len(unique_keywords))

# Create DataFrame
df = pd.DataFrame({
    'Sentence_Index': steps,
    'Unique_Keywords_Count': unique_counts
})

# Display DataFrame
display(df.head(10))

Processing Sentences: 100%|██████████| 1227/1227 [02:39<00:00,  7.70it/s]


,Sentence_Index,Unique_Keywords_Count
0,1,4
1,2,5
2,3,10
3,4,12
4,5,17
5,6,21
6,7,26
7,8,28
8,9,30
9,10,32


In [ ]:
df.tail(30)

,Sentence_Index,Unique_Keywords_Count
1197,1198,881
1198,1199,881
1199,1200,881
1200,1201,881
1201,1202,881
1202,1203,881
1203,1204,881
1204,1205,881
1205,1206,881
1206,1207,881


In [ ]:
import plotly.express as px

# Create Plotly DataFrame plot
fig = px.line(
    df,
    x='Sentence_Index',
    y='Unique_Keywords_Count',
    title='Keyword Accumulation Curve (Species Analogy)',
    labels={
        'Sentence_Index': 'Sentence (Chunk) Index',
        'Unique_Keywords_Count': 'Cumulative Unique Keywords'
    },
    markers=True
)

# Customize layout for zoom/pan and better readability
fig.update_layout(
    template='plotly_white',
    width=1000,
    height=500,
    xaxis=dict(
        tickmode='auto',
        showgrid=True,
        rangeslider=dict(visible=True),  # Adds the draggable slider
        type='linear'
    ),
    yaxis=dict(
        tickmode='linear',
        dtick=5,
        showgrid=True
    )
)

fig.show()


In [ ]:
import plotly.express as px

# Create interactive Plotly plot
fig = px.line(
    df,
    x='Sentence_Index',
    y='Unique_Keywords_Count',
    title='Keyword Accumulation Curve (Species Analogy)',
    labels={
        'Sentence_Index': 'Sentence (Chunk) Index',
        'Unique_Keywords_Count': 'Cumulative Unique Keywords'
    },
    markers=True
)

# Manual tick settings
fig.update_layout(
    template='plotly_white',
    width=1100,
    height=900,
    xaxis=dict(
        tickmode='linear',
        dtick=50,  # Every 100 sentence chunks
        range=[0, 1100],
        rangeslider=dict(visible=True)
    ),
    yaxis=dict(
        tickmode='linear',
        dtick=10,  # Every 10 keywords
        range=[0, 435]
    )
)

fig.show()


In [ ]:
from nltk.tokenize import word_tokenize
with open(filename, 'r') as f:
    transcript = f.read()

# Tokenize words
words = word_tokenize(transcript)
chunk_size = 250

# Generate 250-word chunks
chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Track keywords
unique_keywords = []
unique_keyword_embeddings = []
chunk_indices = []
unique_counts = []

def is_unique(keyword, existing_embeddings, threshold=0.8):
    keyword_emb = embedding_model.encode([keyword])
    if not existing_embeddings:
        return True
    sims = cosine_similarity(keyword_emb, existing_embeddings)
    return sims.max() < threshold

# Loop through each chunk
for i, chunk in enumerate(tqdm(chunks, desc="Processing Chunks")):
    keywords = kw_model.extract_keywords(chunk, top_n=3, stop_words='english')
    for kw, _ in keywords:
        if is_unique(kw, unique_keyword_embeddings):
            unique_keywords.append(kw)
            unique_keyword_embeddings.append(embedding_model.encode(kw))
    chunk_indices.append(i + 1)
    unique_counts.append(len(unique_keywords))

# Create DataFrame
df_chunks_200 = pd.DataFrame({
    'Chunk_Index': chunk_indices,
    'Unique_Keywords_Count': unique_counts
})


Processing Chunks: 100%|██████████| 72/72 [00:28<00:00,  2.49it/s]


In [ ]:
# Plot using Plotly
fig = px.line(
    df_chunks_200,
    x='Chunk_Index',
    y='Unique_Keywords_Count',
    title='Keyword Accumulation Curve (250-Word Chunks)',
    labels={
        'Chunk_Index': '250-Word Chunk Index',
        'Unique_Keywords_Count': 'Cumulative Unique Keywords'
    },
    markers=True
)

fig.update_layout(
    template='plotly_white',
    width=1100,
    height=600,
    xaxis=dict(
        tickmode='linear',
        dtick=1,  # Show every chunk
        rangeslider=dict(visible=True)
    ),
    yaxis=dict(
        tickmode='linear',
        dtick=10
    )
)

fig.show()


In [ ]:
from nltk.tokenize import word_tokenize
with open(filename, 'r') as f:
    transcript = f.read()

# Tokenize words
words = word_tokenize(transcript)
chunk_size = 600

# Generate 600-word chunks
chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Track keywords
unique_keywords = []
unique_keyword_embeddings = []
chunk_indices = []
unique_counts = []

def is_unique(keyword, existing_embeddings, threshold=0.8):
    keyword_emb = embedding_model.encode([keyword])
    if not existing_embeddings:
        return True
    sims = cosine_similarity(keyword_emb, existing_embeddings)
    return sims.max() < threshold

# Loop through each chunk
for i, chunk in enumerate(tqdm(chunks, desc="Processing Chunks")):
    keywords = kw_model.extract_keywords(chunk, top_n=3, stop_words='english')
    for kw, _ in keywords:
        if is_unique(kw, unique_keyword_embeddings):
            unique_keywords.append(kw)
            unique_keyword_embeddings.append(embedding_model.encode(kw))
    chunk_indices.append(i + 1)
    unique_counts.append(len(unique_keywords))

# Create DataFrame
df_chunks_600 = pd.DataFrame({
    'Chunk_Index': chunk_indices,
    'Unique_Keywords_Count': unique_counts
})


Processing Chunks: 100%|██████████| 35/35 [00:18<00:00,  1.87it/s]


In [ ]:
# Plot using Plotly
fig = px.line(
    df_chunks_600,
    x='Chunk_Index',
    y='Unique_Keywords_Count',
    title='Keyword Accumulation Curve (600-Word Chunks)',
    labels={
        'Chunk_Index': '600-Word Chunk Index',
        'Unique_Keywords_Count': 'Cumulative Unique Keywords'
    },
    markers=True
)

fig.update_layout(
    template='plotly_white',
    width=1100,
    height=600,
    xaxis=dict(
        tickmode='linear',
        dtick=1,  # Show every chunk
        rangeslider=dict(visible=True)
    ),
    yaxis=dict(
        tickmode='linear',
        dtick=10
    )
)

fig.show()
